In [1]:
library(tidyverse)
library(here)
library(glue)
library(furrr)
suppressPackageStartupMessages(library(rtracklayer))
devtools::load_all(".")

plan(multiprocess, workers = 20)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /mnt/expressions/mp/archaic-ychr

Attaching package: ‘glue’

The following object is masked from ‘package:dplyr’:

    collapse

Loading required package: future
Loading ychr


# Full 6.9 Mb capture

In [2]:
samples_full <- list.files(here("data/vcf"), "full_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged", negate = T) %>%
    str_replace("full_(.*).vcf.gz", "\\1")

In [3]:
samples_full

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den4_snpad"        "den4"             
 [7] "den8_snpad"        "den8"              "mez2_snpad"       
[10] "mez2"              "S_BedouinB-1"      "S_Burmese-1"      
[13] "S_Dai-2"           "S_Dinka-1"         "S_Finnish-2"      
[16] "S_French-1"        "S_Gambian-1"       "S_Han-2"          
[19] "S_Ju_hoan_North-1" "S_Karitiana-1"     "S_Mandenka-1"     
[22] "S_Mbuti-1"         "S_Papuan-2"        "S_Punjabi-1"      
[25] "S_Saami-2"         "S_Sardinian-1"     "S_Thai-1"         
[28] "S_Turkish-1"       "S_Yoruba-2"        "shotgun_mez2"     
[31] "shotgun_spy1"      "spy1_snpad"        "spy1"             
[34] "ustishim"

In [4]:
bed_full <- here("data/coord/capture_full.bed")

In [5]:
capture_full <- import.bed(bed_full)

In [6]:
sum(width(capture_full))

[1] 6912728

In [7]:
counts_full <- future_map_dfr(samples_full, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_full)
    gt_nodmg <- read_vcf(here(glue("data/vcf/full_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_full)

    tibble(
        individual = ind,
        total = sum(width(capture_full)),
        all = nrow(gt_all),
        all_prop = all / total,
        nodmg = nrow(gt_nodmg),
        nodmg_prop = nodmg / total
    )
})

In [8]:
full_archaic_df <- counts_full %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun)"))
full_modern_df <- counts_full %>% filter(!str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|chimp|shotgun)"))

In [9]:
full_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.1f%%", 100 * .)) %>%
    select(-total) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F))

name,all,all_prop,nodmg,nodmg_prop
<chr>,<chr>,<chr>,<chr>,<chr>
Spy 94a (shotgun),"2,664,787",38.5%,"2,636,405",38.1%
Spy 94a,"3,502,380",50.7%,"3,470,671",50.2%
Spy 94a (snpAD),"3,557,124",51.5%,"3,522,759",51.0%
Mezmaiskaya 2 (shotgun),"3,822,106",55.3%,"3,762,768",54.4%
Denisova 4,"4,731,302",68.4%,"4,705,860",68.1%
Denisova 4 (snpAD),"4,824,242",69.8%,"4,798,037",69.4%
Denisova 8,"5,851,332",84.6%,"5,828,356",84.3%
Denisova 8 (snpAD),"6,260,242",90.6%,"6,236,272",90.2%
Mezmaiskaya 2,"6,348,948",91.8%,"6,346,684",91.8%


In [10]:
full_modern_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total) %>%
    filter(!individual %in% c("den_snpad", "den")) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F))

name,all,all_prop,nodmg,nodmg_prop
<chr>,<chr>,<chr>,<chr>,<chr>
A00-1,"6,852,077",99.12%,"6,851,175",99.11%
A00-2,"6,867,616",99.35%,"6,866,716",99.33%
A00,"6,873,078",99.43%,"6,872,184",99.41%
Ust'-Ishim,"6,888,071",99.64%,"6,887,986",99.64%
S_Yoruba-2,"6,897,101",99.77%,"6,896,820",99.77%
S_Mandenka-1,"6,898,162",99.79%,"6,897,879",99.79%
S_Mbuti-1,"6,898,694",99.80%,"6,898,400",99.79%
S_Gambian-1,"6,899,210",99.80%,"6,898,929",99.80%
S_Papuan-2,"6,904,393",99.88%,"6,904,208",99.88%


# 560 kb capture of regions from Lippold et al.

In [11]:
samples_lippold <- list.files(here("data/vcf"), "lippold_.*.vcf.gz$") %>%
    str_subset("mez2_dp|modern|merged|elsidron2_dp", negate = T) %>%
    str_replace("lippold_(.*).vcf.gz", "\\1")

In [12]:
samples_lippold

[1] "a00_1"             "a00_2"             "a00"              
 [4] "chimp"             "den4"              "den8"             
 [7] "elsidron2"         "mez2"              "S_BedouinB-1"     
[10] "S_Burmese-1"       "S_Dai-2"           "S_Dinka-1"        
[13] "S_Finnish-2"       "S_French-1"        "S_Gambian-1"      
[16] "S_Han-2"           "S_Ju_hoan_North-1" "S_Karitiana-1"    
[19] "S_Mandenka-1"      "S_Mbuti-1"         "S_Papuan-2"       
[22] "S_Punjabi-1"       "S_Saami-2"         "S_Sardinian-1"    
[25] "S_Thai-1"          "S_Turkish-1"       "S_Yoruba-2"       
[28] "spy1"              "ustishim"

In [13]:
bed_lippold <- here("data/coord/capture_lippold.bed")

In [14]:
capture_lippold <- import.bed(bed_lippold)

In [15]:
sum(width(capture_lippold))

[1] 556259

In [16]:
counts_lippold <- future_map_dfr(samples_lippold, function(ind) {
    gt_all <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, bed_filter = bed_lippold)
    gt_nodmg <- read_vcf(here(glue("data/vcf/lippold_{ind}.vcf.gz")), mindp = 3, maxdp = 0.98, nodmg = T, bed_filter = bed_lippold)

    tibble(
        individual = ind,
        total = sum(width(capture_lippold)),
        all = nrow(gt_all),
        all_prop = all / total,
        nodmg = nrow(gt_nodmg),
        nodmg_prop = nodmg / total
    )
})

In [17]:
lippold_archaic_df <- counts_lippold %>% filter(str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|shotgun|elsidron)"))
lippold_modern_df <- counts_lippold %>% filter(!str_detect(individual, "^([Ss]py|[Dd]en|[Mm]ez|chimp|shotgun|elsidron)"))

In [18]:
lippold_archaic_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total) %>%
    filter(!individual %in% c("den_snpad", "den")) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F))

name,all,all_prop,nodmg,nodmg_prop
<chr>,<chr>,<chr>,<chr>,<chr>
Spy 94a,"275,626",49.55%,"272,798",49.04%
Denisova 4,"408,775",73.49%,"406,508",73.08%
Denisova 8,"484,377",87.08%,"482,608",86.76%
Mezmaiskaya 2,"509,145",91.53%,"508,962",91.50%
El Sidrón 1253 (560 kb),"530,172",95.31%,"530,046",95.29%


In [19]:
lippold_modern_df %>%
    arrange(all) %>%
    mutate_at(vars(contains("prop")), ~ sprintf("%.2f%%", 100 * .)) %>%
    select(-total) %>%
    filter(!individual %in% c("den_snpad", "den")) %>%
    mutate(name = fix_name(individual)) %>%
    select(name, everything(), -individual) %>%
    mutate(all = format(all, big.mark = ",", scientific = F),
           nodmg = format(nodmg, big.mark = ",", scientific = F))

name,all,all_prop,nodmg,nodmg_prop
<chr>,<chr>,<chr>,<chr>,<chr>
A00-1,"552,557",99.33%,"552,456",99.32%
S_Yoruba-2,"553,203",99.45%,"553,112",99.43%
S_Mandenka-1,"553,348",99.48%,"553,257",99.46%
S_Gambian-1,"553,365",99.48%,"553,275",99.46%
S_Mbuti-1,"553,381",99.48%,"553,288",99.47%
A00-2,"553,422",99.49%,"553,321",99.47%
A00,"553,618",99.53%,"553,517",99.51%
Ust'-Ishim,"554,575",99.70%,"554,550",99.69%
S_Thai-1,"555,830",99.92%,"555,787",99.92%
